In [7]:
def delta_sigma(R):
    # not R is in kpc
    A = 3.8
    alpha = -.76
    return ((R/1000)**alpha) * A

In [ ]:
def main():
    # note : delta_sigma (R) = sigma(R)
    # surface mass density
    # in paper R_BGS = 40 kpc
    # in paper R_LRG = 70 kpc
    delta_sigma_R_BGS = delta_sigma(40)
    delta_sigma_R_LRG = delta_sigma(70)
    # in paper sigma_R_BGS --> 70 hM_s / pc^2
    # in paper sigma_R_LRG --> 145 hM_s / pc^2
    correction_BGS = 2# should be 1.6
    correction_LRG = # should be more than 1.6
    sigma_R_BGS = delta_sigma_R_BGS * correction_BGS
    print(f"BGS:{sigma_R_BGS}")
    sigma_R_LRG = delta_sigma_R_LRG * correction_LRG
    print(f"LRG:{sigma_R_LRG}")

In [9]:
main()

BGS:87.7501765396766
LRG:57.35074737778642
